In [1]:
from os.path import join
import pandas as pd
import torch
from tqdm.auto import tqdm

path = join("data", "wust-train.json.gz")

df = pd.read_json(path)
df.head()

,"('text',)","('emotions',)"
100,"Lewica at its finest...I nie, nie popieram Kac...",wstret
1000,#poland #polska Bogaci uciekają biednym: http...,neutralne
1001,#braun2020 #PrawyboryKonfederacji #konfederacj...,neutralne
1002,"Dr Andrzej #Anusz: Widać jednoznacznie, że ta ...",neutralne
1003,#Urodziny #lifestyle #refleksja #życie #Polakp...,neutralne


In [2]:
from transformers import XLMTokenizer, RobertaModel

tokenizer = XLMTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")

In [3]:
model = model.to("cuda")

embeddings = []

for emotion, text in tqdm(zip(df["('emotions',)"], df["('text',)"])):
    encoded_input = tokenizer.encode(text, return_tensors='pt')
    encoded_input = encoded_input.to("cuda")
    with torch.no_grad():
        outputs = model(encoded_input)
    embeddings.append((text, outputs[0].detach().cpu().numpy(), emotion))

In [4]:
import pickle
save_path = join("data", "wust_train_word_level_embeddings.pickle")

with open(save_path, 'wb') as handle:
    pickle.dump((embeddings), handle)